In [2]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_c1cbgsrq20/croot/scikit-learn_1690978919468/work


In [3]:
!python -V

Python 3.9.18


In [4]:
import pickle
import pandas as pd
import numpy as np
import os

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/opt/anaconda3/envs/ml-zoomcamp/lib/python3.9/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/envs/ml-zoomcamp/lib/python3.9/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

Q1 : standard deviation of prediction results

In [9]:
print(f"Standar deviation of prediction: {np.std(y_pred):.3f}")

Standar deviation of prediction: 6.247


In [10]:
year = 2023
month = 3

In [11]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [12]:
df_result = pd.DataFrame()
df_result["ride_id"] = df['ride_id']
df_result["predicted_duration"] = y_pred

In [18]:
output_file = f'outputfile/{year:04d}_{month:02d}_result.parquet'

In [19]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [20]:
file_size = os.path.getsize(output_file)
file_size_MB = file_size / (1024 * 1024)
print(f"The size of the file is: {file_size_MB:.2f} MB")

The size of the file is: 65.46 MB
